In [1]:
# TODO Update to V4 of the catalogue...

In [2]:
from imports import *

In [3]:
# FIXED ISO THRESHOLD FOR ALL THE IMAGES (in per pixel units) 
ISOTHRESHOLD = 100

In [ ]:
galaxyname = 'ngc0628'
galaxyname_hst = 'ngc628c'
galaxyname_cat = 'ngc628'

mycat = catalogue_pipeline.PyHSTHACat(galaxyname, galaxyname_hst, galaxyname_cat, output_dir_append='_fixediso_10pc')
mycat.hstha_err = ISOTHRESHOLD
# mycat.rerun_all = True
mycat.hstha_file = mycat.hstha_file.replace('.fits', '_10pc_fixednoise.fits')

# mycat.make_paths()
mycat.load_files()
mycat.get_regions()
mycat.get_cutouts()
mycat.make_catalogue()

# mycat.rerun_cutouts_associations = True
# mycat.make_associations()

In [ ]:
mycat.props['region_ID', 'flag_touch_hst']

In [ ]:
hdus = cat_misc.load_pickle(f'{mycat.cutout_dir}/hdus_all_withmasked.pickel')

In [ ]:
ii = []

for i in range(len(hdus['hstha_hdu_masked_ones'])):
    if np.isnan(hdus['hstha_hdu_masked_ones'][i].data).any()*1:
        sum_ = np.nansum(hdus['hstha_hdu_masked_ones'][i].data[hdus['hstha_hdu_masked_ones'][i].data!=1])
        if sum_!=0:
            ii = ii + [i]

print(len(ii))

In [45]:
edge_mask = mycat.props['flag_touch_hst'] == 1
props_edge = mycat.props[edge_mask]

In [65]:
from scipy.ndimage import binary_closing, binary_fill_holes, binary_erosion, binary_opening, binary_dilation
from matplotlib import pyplot as plt

In [ ]:


iii = int(props_edge['region_ID'][15])

data = hdus['hstha_hdu'][iii].data
mask = ~np.isnan(data)
mask = mask*1

mask_holes = binary_closing(mask, structure=np.ones((3,3)), iterations=5, border_value=1)
# mask_holes = binary_opening(mask, structure=np.ones((3,3)), iterations=5)
# mask_holes = binary_fill_holes(mask, structure=np.ones((1,1)))

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax = ax.ravel()

ax[0].imshow(data)
ax[1].imshow(mask, vmin=0, vmax=1)
ax[2].imshow(mask_holes, vmin=0, vmax=1)
ax[3].imshow(mask-mask_holes, vmin=-1, vmax=1)

In [ ]:
hdu_mask = fits.open('/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc628c/catalogue_fixediso_10pc/ngc0628_mask.fits')[0]
hdu_mask.writeto('ngc0628_mask.fits')
data_mask = hdu_mask.data
data_mask_binary = data_mask > -1
plt.imshow(data_mask_binary)

In [79]:
# region_ID = mycat.props['region_ID'][0]
# region_ID = 2181
region_ID = 649

data_mask_region = data_mask == region_ID
data_mask_region_inv = (data_mask != region_ID) & (data_mask != -1)

data_mask_region = data_mask_region.astype(int)
data_mask_region_inv = data_mask_region_inv.astype(int)

hdu_mask_region = fits.PrimaryHDU(data_mask_region, header=hdu_mask.header)
hdu_mask_region_inv = fits.PrimaryHDU(data_mask_region_inv, header=hdu_mask.header)

hdu_mask_region.writeto('ngc0628_mask_region.fits', overwrite=True)
hdu_mask_region_inv.writeto('ngc0628_mask_region_inv.fits', overwrite=True)

mask_touch = (binary_dilation(data_mask_region) & data_mask_region_inv)
mask_touch = np.sum(mask_touch) > 0 
mask_touch